## Dataset MNIST

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


100%|██████████| 9912422/9912422 [00:00<00:00, 146241958.13it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 68593258.11it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 46482450.49it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 16422869.63it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## Desain Pemilihan Konfigurasi Model

In [21]:
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets

# Baseline Model
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Wide Model
class WideModel(nn.Module):
    def __init__(self):
        super(WideModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 256) # Lebih lebar
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Deep Model
class DeepModel(nn.Module):
    def __init__(self):
        super(DeepModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128) # Lebih dalam
        self.fc4 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

## Looping untuk training model

In [22]:
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

## Evaluasi Model

In [23]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

In [24]:
if __name__ == "__main__":
    baseline_model = BaselineModel()
    wide_model = WideModel()
    deep_model = DeepModel()

    # Pelatihan Model
    num_epochs = 15
    criterion = nn.CrossEntropyLoss()
    optimizer_baseline = optim.Adam(baseline_model.parameters(), lr=0.001)
    optimizer_wide = optim.Adam(wide_model.parameters(), lr=0.001)
    optimizer_deep = optim.Adam(deep_model.parameters(), lr=0.001)

    print("Baseline Model: ")
    train_model(baseline_model, train_loader, criterion, optimizer_baseline, num_epochs)
    print("")

    print("Wide Model: ")
    train_model(wide_model, train_loader, criterion, optimizer_wide, num_epochs)
    print("")

    print("Deep Model: ")
    train_model(deep_model, train_loader, criterion, optimizer_deep, num_epochs)
    print("")

    # Evaluasi Model
    accuracy_baseline = evaluate_model(baseline_model, test_loader)
    accuracy_wide = evaluate_model(wide_model, test_loader)
    accuracy_deep = evaluate_model(deep_model, test_loader)

    # Penjelasan Hasil
    print(f'Accuracy - Baseline Model: {accuracy_baseline}')
    print(f'Accuracy - Wide Model: {accuracy_wide}')
    print(f'Accuracy - Deep Model: {accuracy_deep}')

Baseline Model: 
Epoch 1, Loss: 0.39421698010202916
Epoch 2, Loss: 0.19271088528957195
Epoch 3, Loss: 0.14113070277660006
Epoch 4, Loss: 0.1138913159314265
Epoch 5, Loss: 0.09561898981802848
Epoch 6, Loss: 0.08416499210489807
Epoch 7, Loss: 0.07255322549135676
Epoch 8, Loss: 0.06611793020418458
Epoch 9, Loss: 0.05855294445063919
Epoch 10, Loss: 0.057163893432566906
Epoch 11, Loss: 0.05184060289213724
Epoch 12, Loss: 0.04816057155854992
Epoch 13, Loss: 0.04346180899853834
Epoch 14, Loss: 0.0429977639825262
Epoch 15, Loss: 0.035814029516123266

Wide Model: 
Epoch 1, Loss: 0.3435114584029166
Epoch 2, Loss: 0.15207365417837113
Epoch 3, Loss: 0.11164240651070945
Epoch 4, Loss: 0.0886775097901077
Epoch 5, Loss: 0.07835332361305121
Epoch 6, Loss: 0.06728839244780892
Epoch 7, Loss: 0.05987028676039998
Epoch 8, Loss: 0.0531685647312929
Epoch 9, Loss: 0.04760867730577363
Epoch 10, Loss: 0.042161036980525056
Epoch 11, Loss: 0.039758235319648616
Epoch 12, Loss: 0.035149737024433546
Epoch 13, Loss:

## Penjelasan performa dari setiap model

### Baseline Model

Model ini adalah model dasar dengan 3 lapisan tersembunyi yang terdiri dari 128 neuron serta meiliki arsitektur 128 -> 64 -> 10, ini mengindikasikan bahwa model memiliki kemampuan yang cukup baik untuk mempelajari pola dalam data, dan pelatihan konvergen pada tingkat loss yang rendah secara signifikan. Akurasi yang dicapai adalah sekitar 97,4%, yang menunjukkan model dapat mengklasifikasikan digit-digit MNIST dengan baik.

### Wide Model

Model ini lebih lebar dibandingkan dengan Baseline model, dengan jumlah neuron yang lebih banyak sebesar 256 neuron dan 3 lapisan tersembunyi serta memiliki arsitektur  256 -> 128 -> 10. Terlihat bahwa model wide mencapai loss yang lebih rendah lebih cepat selama epoch pertama dibandingkan dengan Baseline model. Model ini mencapai akurasi 97.19%. Perbedaan ini menunjukkan bahwa tambahan neuron dapat membantu percepatan konvergensi.

### Deep Model

Model ini lebih dalam dibandingkan dengan Baseline model, dengan empat lapisan tersembunyi dan jumlah neuron sebesar 128 neuron serta memiliki arsitektur 128 -> 128 -> 128 -> 10.  Loss model deep menurun secara signifikan selama epoch kedua, menunjukkan kemampuan model untuk memahami pola yang lebih kompleks dalam data, namun pada beberapa epoch berikutnya, penurunan loss menjadi lebih lambat dibandingkan dengan model lainnya. Akurasi yang dicapai oleh model deep adalah sekitar 96.52. Ini menunjukkan bahwa kedalaman model dapat membantu dalam pemahaman yang lebih baik terhadap data, meskipun pelatihan memerlukan beberapa epoch lebih lama.

## Kesimpulan

Baseline Model tetap memiliki performa yang baik, terutama mengingat kesederhanaan arsitekturnya, sedangkan Model wide dan deep masing-masing memiliki kelebihan dan kekurangan. Model wide cenderung konvergen lebih cepat dengan akurasi yang baik, sementara model deep memiliki kemampuan untuk memahami pola yang lebih kompleks dalam data, meskipun memerlukan lebih banyak epoch untuk konvergensi dengan waktu yang lebih lama.

Kesimpulannya, pemilihan arsitektur model tergantung pada kebutuhan dan sumber daya yang tersedia. Model wide cocok untuk aplikasi yang memerlukan pelatihan cepat, sedangkan model deep cocok untuk tugas yang memerlukan pemahaman yang lebih mendalam terhadap data. Jadi, dari interpretasi di atas, Model Deep adalah pilihan terbaik karena memiliki akurasi tertinggi dan kinerja yang baik secara keseluruhan dalam mengklasifikasikan data MNIST.